## [Neural Network 10] 다층 퍼셉트론 구현 실습

https://www.youtube.com/watch?v=fcoVlBIYD54&list=PLfGJDDf2OqlSAL9kE4FvT_rG4DH_8S4AQ&index=5




> $ (x_1) \qquad → w_1=0.7 \ (z_1 | h_1) $ <br>
> $ \quad  ↘ w_2 = 0.3 \qquad\qquad\qquad ↘ w_5 = 0.55 $  <br>
> $ \qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad (z_3|o_1) $ <br>
> $ \quad  ↗ w_3 = 0.4 \qquad\qquad\qquad ↗ w_6 = 0.45 $  <br>
> $ (x_2) \qquad → w_4=0.6 \ (z_2 | h_2) $ <br>



- 항목이 1개 뿐일 때의 손실 함수 계산
  - $ C = \frac{1}{1} \sum_{i}^{1} (y_1 - \hat{y}_1)^2 $
  - 
  - => `np.mean((y_true - y_pred) ** 2)
`
- ...

In [1]:
import numpy as np

In [7]:
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # 가중치 초기화
        self.w1_2_3_4 = np.random.random((self.input_size, self.hidden_size))
        # self.w1_2_3_4 = [[1, 10], [1, 10]]
        self.w5_6 = np.random.random((self.hidden_size, self.output_size))
        # self.w5_6 = [[-40], [40]]  # 2x1

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, X):
        # X: 입력 샘플 하나. 1x2
        # propagate inputs through the network
        self.z1_2 = np.dot(X, self.w1_2_3_4)   # (1,2) dot (2,2) => (1,2)
        '''
            X = [[x1 x2]]
            z1 = x1w1 + x2w3
            z2 = x1w2 + x2w4
            z1_2 = [[z1 z2]]
                 = [[x1w1+x2w3 x1w2+x2w4]]
                 = [[x1 x2]]@[[w1 w2]
                              [w3 w4]]
                    # X가 1D가 아니고 2D이므로 @로 동작함. 즉
                    # (1,2)@(2,2) => (1,2)
                 =    X @ w_1_2_3_4
                 = np.dot(X, w_1_2_3_4)
            w1_2_3_4 = [[w1 w2]
                        [w3 w4]]
        '''
        self.h1_2 = self.sigmoid(self.z1_2)    # (1,2)
        '''
            h1 = sigmoid(z1)
            h2 = sigmoid(z2)
            h1_2 = [[h1 h2]]
                 = [[sigmoid(z1) sigmoid(z2)]]
                 = sigmoid([[z1 z2]])  # <- sigmoid is arithmetic operations.
                 = sigmoid(z1_2)
        '''
        self.z3 = np.dot(self.h1_2, self.w5_6) # (1,2) dot (2,1) => (1,1)
            # 이 경우 numpy는 차원을 축소하여 그냥 무차원 스칼라 값을 리턴한다. shape:()
        '''
            z3 = [[h1w5 + h2w6]]
               = [[h1 h2]]·[[w5]
                            [w6]]
               = h1_2 · w5_6
        '''
        self.o1 = self.sigmoid(self.z3)   # 1x1
        '''
            o1 = sigmoid(z3)
        '''
        return self.o1  # 스칼라 값이 아닌 (1,1)행렬이다.

    def mse_loss(self, y_true, y_pred):
        # MSE 손실 계산
        return np.mean((y_true - y_pred) ** 2)

    def backward(self, X, y, y_pred, learning_rate):
        # 각 인자의 shape: X는 (1,2), y는 (1,1), y_pred는 (1,1)
        # 체인룰 계산.
        # 출력 층
        dc_do1 = -2 * (y - y_pred)  # (1,1)
            # MSE의 미분. 출력증이 1개 뿐이라서 간단.
            # dc/do1 = d((y-o1)^2)/do1 = -2(y-o1)
        do1_dz3 = y_pred * (1 - y_pred) # (1,1)
            # sigmoid의 미분.  O'(z) = O(z)(1-O(z))
        dz3_dw5_6 = self.h1_2  # (1,2)
            # dz3/dw5 = d(h1w5 + h2w6)/dw5 = h1
            # dz3/dw6 = d(h1w5 + h2w6)/dw6 = h2
        dc_dw5_6 = dc_do1 * do1_dz3 * dz3_dw5_6  # all (1,2)

        # 학습
        self.w5_6 = self.w5_6 + learning_rate * -dc_dw5_6.T
        '''
            w5 = w5 - lr * dc_dw5
            w6 = w6 - lr * dc_dw6
                다른 x1_2, z1_2 와 달리 이 w5_6 은 1x2 가 아니고 2x1 이다.
                일부러 이렇게 한 것으로 보임.
            [[w5] = [[w5] - lr * [[dc_dw5]
             [w6]]   [w6]]        [dc_dw6]]
        '''

        # 은닉층
        dc_dw1_2_3_4 = dc_do1 * do1_dz3 * \
            np.dot(self.w5_6 * (self.h1_2 * (1 - self.h1_2)).T, X)
            # 왜 여기에 .T 가 필요한가??
        '''
            dc_dw1_2_3_4 = [[dc_dw1 dc_dw2]
                            [dc_dw3 dc_dw4]]

            % w1~w4 까지 각 요소 별 계산
            dc_dw1 = dc_do1 * do1_dz3 * dz3_dh1 * dh1_dz1 * dz1_dw1
                    앞의 두개는 출력층에서 이미 구한 값이고, w1~4와 무관한 값.
                      이를 A, B 라고 하자. 둘 다 (1,2) shape 이다.
                    뒤의 셋은 w1~w4 에 따라 식이 다름.
                dz3_dh1 = w5
                dh1_dz1 = sigmoid(z1) x (1 - sigmoid(z1)) = h1(1-h1) = H1
                    % note that h1 = sigmoid(z1)
                dz1_dw1 = x1
            dc_dw1 = A B w5 H1 x1
            dc_dw2 = A B w6 H2 x1
            dc_dw3 = A B w5 H1 x2
            dc_dw4 = A B w6 H2 x2

            !!!! 주의 !!!!
            # 아래 식에서 보겠지만 w2 와 w3의 위치가 바뀌어 정의를 해야 이 식이 된다.
            dc_dw1_2_3_4 = [[ dc_dw1 dc_dw3 ]
                            [ dc_dw2 dc_dw4 ]]

            dc_dw1_2_3_4 = A B [[ w5H1x1  w5H1x2 ]
                                [ w6H2x1  w6H2x2 ]]
                         = A B [[ w5H1 ] @[[x1 x2]]
                                [ w6H2 ]]
                         = A B [[w5] * [[H1]  @ [[x1 x2]]
                                [w6]]   [H2]]
                         = A B w5_6 H.T @ X
        '''

        # 학습
        self.w1_2_3_4 = self.w1_2_3_4 + learning_rate * -dc_dw1_2_3_4.T
        '''
        w1_2_3_4 와 dc_dw1_2_3_4 의 배치를 일치시키지 않았기 때문에
        dc_dw1_2_3_4 에 transpose를 적용하고 있음.

        '''

    def train(self, X_train, y_train, epochs, learning_rate):
        # X_train의 shape은 (100,2)
        # y_train의 shape은 (100,)

        # for epoch in range(epochs):
        for epoch in range(epochs):
            for i in range(len(X_train)): # 입력 샘플 수 만큼
                # forward pass
                y_pred = self.forward([X_train[i]])
                    # 그냥 X_train[i] 이면 (2,) 벡터일텐데, [ ]를 씌워서 (1,2) 행렬로 만들어 forward()에 전달.
                    # 리턴값 y_pred 는 1x1 행렬

                # compute and print loss
                loss = self.mse_loss([y_train[i]], y_pred)

                # backward pass
                self.backward([X_train[i]], [y_train[i]], y_pred, learning_rate)
                    # 전달하는 데이터의 shape: (1,2), (1,1), (1,1), ()

            if np.mod(epoch, 100) == 0:
                print('epoch=', epoch, 'loss=', loss)


In [8]:
# 데이터 생성
X_train = np.random.randint(0, 2, (100,2))
# shape 지정에 의해 100x2 랜덤 행렬 생성. 0 또는 1 로만 구성됨.

# XOR 게이트. 두 값이 다르면 true -> 1
y_train = (X_train[:,0] != X_train[:,1]).astype(int)

In [9]:
# 생성된 데이터 확인. 앞의 3개만..
print(X_train[:3])
print(y_train[:3])
print(type(y_train), y_train.shape)

[[1 0]
 [0 0]
 [1 1]]
[1 0 0]
<class 'numpy.ndarray'> (100,)


In [10]:
# 다층 퍼셉트론 선언
mlp = MLP(input_size=2, hidden_size=2, output_size=1)

In [11]:
# 모델 학습
mlp.train(X_train, y_train, epochs=1000, learning_rate=0.1)

epoch= 0 loss= 0.27984034874769653
epoch= 100 loss= 0.04843941681583439
epoch= 200 loss= 0.061079411325569204
epoch= 300 loss= 0.04668928828527995
epoch= 400 loss= 0.028393059761331656
epoch= 500 loss= 0.02037373157208337
epoch= 600 loss= 0.015944864686015914
epoch= 700 loss= 0.013140929216945305
epoch= 800 loss= 0.01120309345011151
epoch= 900 loss= 0.009780428760563554


In [12]:
# 테스트 값으로 모델 값 예측
test_input = np.array([[0, 0]])
predicted_output = mlp.forward(test_input)
print('Predicted Output:', test_input, predicted_output)

test_input = np.array([[1, 0]])
predicted_output = mlp.forward(test_input)
print('Predicted Output:', test_input, predicted_output)

test_input = np.array([[0, 1]])
predicted_output = mlp.forward(test_input)
print('Predicted Output:', test_input, predicted_output)

test_input = np.array([[1, 1]])
predicted_output = mlp.forward(test_input)
print('Predicted Output:', test_input, predicted_output)

Predicted Output: [[0 0]] [[0.05747596]]
Predicted Output: [[1 0]] [[0.87222297]]
Predicted Output: [[0 1]] [[0.91097168]]
Predicted Output: [[1 1]] [[0.14241516]]
